## ComCam Image Header Check

In [ ]:
import logging
import re
import sys

from astropy.time import Time, TimeDelta
import numpy as np

import lsst.daf.butler as dafButler
from lsst_efd_client import EfdClient
from lsst.ts.idl.enums import MTPtg as eMTPtg

from check_utils import check, check_float, check_near, check_not_empty

run_logging = False

In [ ]:
if run_logging:
    stream_handler = logging.StreamHandler(sys.stdout)

    logger = logging.getLogger()
    logger.addHandler(stream_handler)
    logger.level = logging.NOTSET
    logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [ ]:
# TTS
dataPath = "/repo/LSSTComCam"
efd_name = "tucson_teststand_efd"
# RSP
#dataPath = "/repo/main"
#efd_name = "ldf_stable_efd"

collections = ["LSSTComCam/raw/all"]
instrument = "LSSTComCam"
butler = dafButler.Butler(dataPath, collections=collections, instrument=instrument)
client = EfdClient(efd_name)

In [ ]:
day_obs = 20220418
seq_num = 17
raft_name = "R22"
detector = 0
dataId = {"instrument": instrument, "detector.raft": raft_name, "detector.id": detector,
          "exposure.day_obs": day_obs, "exposure.seq_num": seq_num}
raw = butler.get('raw.metadata', dataId)

In [ ]:
header = raw.toDict()
#print(list(header.keys()))
#print(header)

In [ ]:
if day_obs == 20220401 and seq_num == 6:
    header["DATE-BEG"] = "2022-04-01T23:56:31.438"
    header["DATE-END"] = "2022-04-01T23:56:32.537"
if day_obs == 20220414 and seq_num == 32:
    header["DATE-BEG"] = "2022-04-14T21:08:36.863"
    header["DATE-END"] = "2022-04-14T21:08:41.922"

In [ ]:
delta = TimeDelta(20.0, format="sec", scale="tai")
obs_beg = Time(header["DATE-BEG"], format="fits", scale="tai")
obs_end = Time(header["DATE-END"], format="fits", scale="tai")
exp_time = TimeDelta(float(header["EXPTIME"]), format="sec", scale="tai")
#print(obs_beg)
#print(header["DATE-BEG"])
obs_id = header["OBSID"]
#print(f"IMGTYPE: {header['IMGTYPE']}")
#print(f"EXPTIME: {header['EXPTIME']}")
#print(f"DARKTIME: {header['DARKTIME']}")
print(f"DATE-BEG: {header['DATE-BEG']}")
print(f"DATE-END: {header['DATE-END']}")

In [ ]:
start_int_df = await client.select_time_series("lsst.sal.CCCamera.logevent_startIntegration", "*",
                                               obs_beg.utc, delta, is_window=True)
end_image_tel_df = await client.select_time_series("lsst.sal.CCCamera.logevent_endOfImageTelemetry", "*",
                                                   obs_end.utc, delta, is_window=True)

In [ ]:
print(f"Start Integration Num Rows: {len(start_int_df)}")
print(f"End of Image Telemetry Num Rows: {len(end_image_tel_df)}")

In [ ]:
start_int = start_int_df.loc[start_int_df["imageName"] == obs_id]
end_image_tel = end_image_tel_df.loc[end_image_tel_df["imageName"] == obs_id]

In [ ]:
# Scales are UTC because incoming timestamps are already TAI and astropy screws them up if the scale is TAI.
date_obs = Time(end_image_tel["timestampDateObs"][0], format="unix_tai", scale="tai")
date_end = Time(end_image_tel["timestampDateEnd"][0], format="unix_tai", scale="tai")
date_start_int = Time(start_int["private_sndStamp"][0], format="unix_tai", scale="tai").utc
date_eoit = Time(end_image_tel["private_sndStamp"][0], format="unix_tai", scale="tai").utc
#print(date_start_int.isot)
#print(date_eoit.isot)

In [ ]:
check("DATE-BEG", header, date_obs.isot)
check("DATE-END", header, date_end.isot)

In [ ]:
date_diff = date_end - date_obs
if date_diff.sec >= float(header["EXPTIME"]):
    print("DATE-END - DATE-BEG OK")
else:
    print(f"Problem with DATE-END - DATE-BEG: {date_diff.sec} seconds")
    print(f"Exposure time: {header['EXPTIME']} seconds")

In [ ]:
image_readout_params_df = await client.select_top_n("lsst.sal.CCCamera.logevent_imageReadoutParameters",
                                                    "*", 1,
                                                    time_cut=date_start_int.isot)

In [ ]:
check("TELCODE", header, start_int["imageSource"][0])
check("CONTRLLR", header, start_int["imageController"][0])
check("DAYOBS", header, start_int["imageDate"][0])
check("SEQNUM", header, start_int["imageNumber"][0])
additional_keys = start_int["additionalKeys"][0].split(":")
additional_values = re.split(r'(?<!\\):', start_int["additionalValues"][0])
check("GROUPID", header, additional_values[additional_keys.index("groupId")].replace("\\", ""))
check("IMGTYPE", header, additional_values[additional_keys.index("imageType")])
check("TESTTYPE", header, additional_values[additional_keys.index("testType")])
check("OBSANNOT", header, image_readout_params_df["daqAnnotation"][0])
check("CURINDEX", header, start_int["imageIndex"][0])
check("MAXINDEX", header, start_int["imagesInSequence"][0])
check_not_empty("TSTAND", header)

In [ ]:
check("OVERH", header, image_readout_params_df[f"overRows{detector}"][0])
check("OVERV", header, image_readout_params_df[f"overCols{detector}"][0])
check("PREH", header, image_readout_params_df[f"preCols{detector}"][0])

In [ ]:
shutter_motion_profile_df = await client.select_top_n("lsst.sal.CCCamera.logevent_endOfImageTelemetry", "*", 1,
                                                      time_cut=date_eoit.isot)

In [ ]:
check_float("EXPTIME", header, start_int["exposureTime"][0])
check_float("DARKTIME", header, end_image_tel["darkTime"][0])
check_float("SHUTTIME", header, shutter_motion_profile_df["measuredShutterOpenTime"][0])

In [ ]:
current_target_df = await client.select_top_n("lsst.sal.MTPtg.logevent_currentTarget", "*", 1,
                                              time_cut=date_start_int.isot)
current_target_status_df = await client.select_top_n("lsst.sal.MTPtg.currentTargetStatus", "*", 1,
                                                     time_cut=date_start_int.isot)
mount_positions_start_df = await client.select_top_n("lsst.sal.MTPtg.mountPosition", "*", 1,
                                                     time_cut=date_start_int.isot)
mount_positions_end_df = await client.select_top_n("lsst.sal.MTPtg.mountPosition", "*", 1,
                                                   time_cut=date_eoit.isot)
mount_az_start_df = await client.select_top_n("lsst.sal.MTMount.azimuth", "*", 1,
                                              time_cut=date_start_int.isot)
mount_az_end_df = await client.select_top_n("lsst.sal.MTMount.azimuth", "*", 1,
                                            time_cut=date_eoit.isot)
mount_el_start_df = await client.select_top_n("lsst.sal.MTMount.elevation", "*", 1,
                                              time_cut=date_start_int.isot)
mount_el_end_df = await client.select_top_n("lsst.sal.MTMount.elevation", "*", 1,
                                            time_cut=date_eoit.isot)
hexapod_app_df = await client.select_top_n("lsst.sal.MTHexapod.application", "*", 1,
                                           time_cut=date_start_int.isot, index=1)

In [ ]:
if header["IMGTYPE"] in ["OBJECT", "ENGTEST"]:
    check("OBJECT", header, current_target_df["targetName"][0])
    check_float("RA", header, np.degrees(current_target_df["ra"])[0])
    check_float("DEC", header, np.degrees(current_target_df["declination"])[0])
    check_float("ROTPA", header, current_target_df["rotPA"][0])
    check("ROTCOORD", header, "sky")
    check_float("RASTART", header, mount_positions_start_df["ra"][0])
    check_float("RAEND", header, mount_positions_end_df["ra"][0])
    check_near("RA", "RASTART", header)
    check_near("RA", "RAEND", header)
    check_float("DECSTART", header, mount_positions_start_df["declination"][0])
    check_float("DECEND", header, mount_positions_end_df["declination"][0])
    check_near("DEC", "DECSTART", header)
    check_near("DEC", "DECEND", header)
else:
    print(f"No check on RA/DEC for {header['IMGTYPE']}")
check_float("AZSTART", header, mount_az_start_df["actualPosition"][0])
check_float("AZEND", header, mount_az_end_df["actualPosition"][0])
check_float("ELSTART", header, mount_el_start_df["actualPosition"][0])
check_float("ELEND", header, mount_el_end_df["actualPosition"][0])
check_float("FOCUSZ", header, hexapod_app_df["demand1"][0])
check("TRACKSYS", header, eMTPtg.TargetTypes(current_target_df["targetType"][0]).name)
check("RADESYS", header, eMTPtg.CoordFrame(current_target_df["frame"][0]).name)

In [ ]:
sim_mtmount_df = await client.select_top_n("lsst.sal.MTMount.logevent_simulationMode", "mode", 1,
                                           time_cut=date_start_int.isot)
sim_mtm1m3_df = await client.select_top_n("lsst.sal.MTM1M3.logevent_simulationMode", "mode", 1,
                                          time_cut=date_start_int.isot)
sim_mtm2_df = await client.select_top_n("lsst.sal.MTM2.logevent_simulationMode", "mode", 1,
                                        time_cut=date_start_int.isot)
sim_mtcamhex_df = await client.select_top_n("lsst.sal.MTHexapod.logevent_simulationMode", "mode", 1,
                                            time_cut=date_start_int.isot, index=1)
sim_mtm2hex_df = await client.select_top_n("lsst.sal.MTHexapod.logevent_simulationMode", "mode", 1,
                                           time_cut=date_start_int.isot, index=2)
sim_mtrotator_df = await client.select_top_n("lsst.sal.MTRotator.logevent_simulationMode", "mode", 1,
                                             time_cut=date_start_int.isot)
sim_mtdome_df = await client.select_top_n("lsst.sal.MTDome.logevent_simulationMode", "mode", 1,
                                          time_cut=date_start_int.isot)
sim_mtdometraj_df = await client.select_top_n("lsst.sal.MTDomeTrajectory.logevent_simulationMode", "mode", 1,
                                              time_cut=date_start_int.isot)

In [ ]:
check("SIMULATE MTMOUNT", header, sim_mtmount_df["mode"][0])
try:
    value = sim_mtm1m3_df["mode"][0]
except KeyError:
    value = "None"
check("SIMULATE MTM1M3", header, value)
check("SIMULATE MTM2", header, sim_mtm2_df["mode"][0])
check("SIMULATE CAMHEXAPOD", header, sim_mtcamhex_df["mode"][0])
check("SIMULATE M2HEXAPOD", header, sim_mtm2hex_df["mode"][0])
check("SIMULATE MTROTATOR", header, sim_mtrotator_df["mode"][0])
check("SIMULATE MTDOME", header, sim_mtdome_df["mode"][0])
check("SIMULATE MTDOMETRAJECTORY", header, sim_mtdometraj_df["mode"][0])

In [ ]:
filter_pos_df = await client.select_top_n("lsst.sal.CCCamera.logevent_endSetFilter", "*", 1,
                                          time_cut=date_start_int.isot)

In [ ]:
check("FILTBAND", header, filter_pos_df["filterType"][0])
check("FILTER", header, filter_pos_df["filterName"][0])
check("FILTSLOT", header, filter_pos_df["filterSlot"][0])
check("FILTPOS", header, filter_pos_df["filterPosition"][0])

In [ ]:
focal_plane_info_df = await client.select_top_n("lsst.sal.CCCamera.logevent_focalPlaneSummaryInfo", "*", 1,
                                                time_cut=date_start_int.isot)
fp_ccd_df = await client.select_top_n("lsst.sal.CCCamera.focal_plane_Ccd", "*", 1,
                                      time_cut=date_start_int.isot)

In [ ]:
check("CCD_MANU", header, focal_plane_info_df["ccdManufacturer"][0], index=detector)
check("CCD_TYPE", header, focal_plane_info_df[f"ccdType{detector}"][0])
check("CCD_SERN", header, focal_plane_info_df["ccdManSerNum"][0], index=detector)
check("LSST_NUM", header, focal_plane_info_df["ccdLSSTName"][0], index=detector)
check("CCDSLOT", header, focal_plane_info_df["ccdSlot"][0], index=detector)
check("RAFTBAY", header, focal_plane_info_df["raftBay"][0], index=detector)
check("SEQCKSUM", header, focal_plane_info_df["sequencerChecksum"][0], index=detector)
check("SEQNAME", header, focal_plane_info_df["sequencerKey"][0], index=detector)
check("REBNAME", header, focal_plane_info_df["rebLSSTName"][0], index=detector)
check("CONTNUM", header, focal_plane_info_df["rebSerialNumber"][0], index=detector)
check_float("CCDTEMP", header, fp_ccd_df[f"temp{detector}"][0])
check_float("TEMP_SET", header, focal_plane_info_df[f"ccdTempSetPoint{detector}"][0])

In [ ]:
check("FACILITY", header, "Vera C. Rubin Observatory")
check_not_empty("DATE", header)
check_not_empty("MJD-OBS", header)
check_not_empty("MJD-BEG", header)
check_not_empty("MJD-END", header)
check("BUNIT", header, "adu")
check("TELESCOP", header, "Simonyi Survey Telescope")
check("INSTRUME", header, "ComCam")
check("OBSERVER", header, "LSST")
check_float("OBS-LONG", header, -70.749417)
check_float("OBS-LAT", header, -30.244639)
check_float("OBS-ELEV", header, 2663.0)
check_float("OBSGEO-X", header, 1818938.94)
check_float("OBSGEO-Y", header, -5208470.95)
check_float("OBSGEO-Z", header, -3195172.08)
check_not_empty("FILENAME", header)
check_not_empty("HEADVER", header)
check_not_empty("TSTAND", header)
check_not_empty("CHECKSUM", header)
check_not_empty("DATASUM", header)
check("TIMESYS", header, "TAI")

In [ ]:
# Segment headers
check("XTENSION", header, "IMAGE")
check("BITPIX", header, 32)
check("INHERIT", header, "T")
check("PCOUNT", header, 0)
check("GCOUNT", header, 1)
check_not_empty("EXTNAME", header)
check_not_empty("DATASEC", header)
check_not_empty("DETSEC", header)
check_not_empty("DTV1", header)
check_not_empty("DTV2", header)
check_not_empty("DTM1_1", header)
check_not_empty("DTM1_2", header)
check_not_empty("DTM2_1", header)
check_not_empty("DTM2_2", header)